In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import matplotlib.pyplot as plt
import numpy as np
import torch
from mpl_toolkits import mplot3d

import crossmodal
import diffbayes
import fannypack

In [91]:
# Choose experiment to eval
# experiment_name = "pf_blackout0.0_new"
# experiment_name = "pf_blackout0.0_nommpretrain"
Task = crossmodal.tasks.DoorTask
experiment_name = "cmpf128_blackout0.0_1e5"

In [92]:
# Create Buddy, model
buddy = fannypack.utils.Buddy(experiment_name)
model_type = buddy.metadata["model_type"]
dataset_args = buddy.metadata["dataset_args"]
filter_model = Task.model_types[model_type]()
buddy.attach_model(filter_model)

[buddy-cmpf128_blackout0.0_1e5] Using device: cuda
[buddy-cmpf128_blackout0.0_1e5] Loaded metadata: {'commit_hash': '6a2db3f8ac63b442313f9704c3f92f6e52f0eb16', 'dataset_args': {'image_blackout_ratio': 0.0, 'sequential_image_rate': 1, 'use_haptics': True, 'use_proprioception': True, 'use_vision': True}, 'model_type': 'DoorCrossmodalParticleFilter', 'notes': '(none)', 'train_start_time': 'Jul 29, 2020 @ 16:20:58'}
[buddy-cmpf128_blackout0.0_1e5] Wrote metadata to: metadata/cmpf128_blackout0.0_1e5.yaml
[buddy-cmpf128_blackout0.0_1e5] Available checkpoint labels: ['0000000000266126', '0000000000267173', '0000000000268222', '0000000000269250', '0000000000270241', 'phase0', 'phase1', 'phase2', 'phase3']


In [93]:
# Load a trajectory
fannypack.data.set_cache_path(crossmodal.__path__[0] + "/../.cache")
trajectories = crossmodal.tasks._door._load_trajectories(
    ("panda_door_push_100.hdf5", 5), **dataset_args
)

[TrajectoriesFile-...oor_push_100.hdf5] Loading trajectory from file: <HDF5 file "1JEDGZWpWE-Z9kuCvRBJh_Auhc-2V0UpN-panda_door_push_100.hdf5" (mode r)>
[TrajectoriesFile-...oor_push_100.hdf5] Existing trajectory count: 100
[TrajectoriesFile-...oor_push_100.hdf5] Opening file...
[TrajectoriesFile-...oor_push_100.hdf5] Closing file...


In [94]:
# Pull out trajectory
states, observations, controls = trajectories[0]

print("States shape:", states.shape)
print("Controls shape:", controls.shape)

States shape: (800, 3)
Controls shape: (800, 7)


In [95]:
span = 6.0
resolution = 0.03

# Time step to visualize at
t = 0

# Create delta
grid_delta = np.mgrid[
    -span / 2.0 : span / 2.0 : resolution, -span / 2.0 : span / 2.0 : resolution
]
_, grid_cols, grid_rows = grid_delta.shape
grid_delta = grid_delta.T.reshape((1, grid_cols * grid_rows, 2))
zero_col = np.zeros((1, grid_cols * grid_rows, 1))
grid_delta = np.concatenate((zero_col, grid_delta), axis=2)
assert grid_delta.shape == (1, grid_cols * grid_rows, 3)

# Get state, observation
state_input = fannypack.utils.to_torch(
    states[t : t + 1, None] + grid_delta, device=buddy.device
)
observation_input = fannypack.utils.to_torch(
    fannypack.utils.SliceWrapper(observations)[t : t + 1], device=buddy.device
)

# Enable grad computation for inputs
for v in observation_input.values():
    v.requires_grad = True

In [96]:
print(observation_input.keys())
observation_input["image"].shape
plt.figure()
plt.imshow(fannypack.utils.to_numpy(observation_input["image"][0]), cmap="gray")
plt.show()

dict_keys(['gripper_pos', 'gripper_sensors', 'image'])


/juno/u/brentyi/miniconda/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [98]:
# Load latest checkpoint
buddy.load_checkpoint()

# Get measurement model
measurement_model = filter_model.measurement_model
model_name = ""
if isinstance(
    filter_model.measurement_model,
    crossmodal.base_models.CrossmodalParticleFilterMeasurementModel,
):
    # Index 0 is image model, index 1 is proprioception/haptics
    index = 1
    model_name = {0: "(image)", 1: "(proprioception + haptics)"}[index]
    measurement_model = measurement_model.measurement_models[index]

# Compute weights
log_weights = measurement_model(states=state_input, observations=observation_input)

[buddy-cmpf128_blackout0.0_1e5] Read checkpoint from path: checkpoints/cmpf128_blackout0.0_1e5-0000000000270241.ckpt
[buddy-cmpf128_blackout0.0_1e5] Loaded checkpoint at step: 270241


In [100]:
filter_model.measurement_model.crossmodal_weight_model(observations=observation_input)

tensor([[20.7841,  0.1160]], device='cuda:0', grad_fn=<ExpBackward>)

In [82]:
def process_weights(log_weights):
    weights = np.exp(fannypack.utils.to_numpy(log_weights))
    weights = weights.reshape((grid_cols, grid_rows)) / np.max(weights)
    return weights


weights = process_weights(log_weights)
# weights_pretrain = process_weights(log_weights_pretrain)

In [83]:
fig = plt.figure()
ax = plt.axes(projection="3d")

x = grid_delta[0, :, 1].reshape((grid_cols, grid_rows))
y = grid_delta[0, :, 2].reshape((grid_cols, grid_rows))

ax.plot_surface(x, y, weights, cmap="viridis", edgecolor="none", alpha=1.0)
# ax.plot_surface(x, y, weights_pretrain, cmap="plasma", edgecolor="none", alpha=0.3)

ax.set_title(f"Hinge X/Y Likelihood Map {model_name}")
ax.set_xlabel("X error")
ax.set_ylabel("Y error")
plt.show()

/juno/u/brentyi/miniconda/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [84]:
fig = plt.figure()
ax = plt.axes(projection="3d")

x = grid_delta[0, :, 1].reshape((grid_cols, grid_rows))
y = grid_delta[0, :, 2].reshape((grid_cols, grid_rows))

ax.plot_surface(x, y, weights, cmap="viridis", edgecolor="none", alpha=1.0)
# ax.plot_surface(x, y, weights_pretrain, cmap="plasma", edgecolor="none", alpha=0.3)

ax.set_title("Door Hinge X/Y Likelihood Map")
plt.show()

/juno/u/brentyi/miniconda/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [30]:
# Compute saliency
torch.mean(log_weights).backward(retain_graph=True)

In [31]:
image, saliency_map = fannypack.utils.to_numpy(
    [observation_input["image"][0, :, :], observation_input["image"].grad[0, :, :]]
)
saliency_map = np.abs(saliency_map)

fig, axes = plt.subplots(1, 2)
axes = axes.flatten()
axes[0].imshow(image)
axes[1].imshow(saliency_map)
fig.show()

TypeError: 'NoneType' object is not subscriptable

In [32]:
print(observation_input["gripper_pos"].grad)
print(observation_input["gripper_sensors"].grad)

tensor([[ 3.3854, -5.8738,  0.4430]], device='cuda:0')
tensor([[ 1.2563, -0.2116, -0.3108, -0.2650, -1.2925,  0.6730, -1.2703]],
       device='cuda:0')
